### Import of modules/libs

In [1]:
import collada as cl
# import pandas as pd
import numpy as np
import math



### Import of data

In [2]:

# pathToFile :str = 'cubes_scene.dae'
pathToFile :str = 'newExportFromBlender.dae'
with open(file=pathToFile, mode="rb") as cubesSceneFile:
    mesh = cl.Collada(cubesSceneFile
                    #   , 
                    #   ignore=[cl.common.DaeUnsupportedError, cl.common.DaeBrokenRefError]
                      )
    
    mesh.assetInfo.upaxis= "Z_UP"
    print("This is the whole scene source list object \(1-cos)",mesh)
    # print(mesh.geometries)
    print("This is the first geometry object \(1-cos)\(1-cos)",mesh.geometries[0])
    print("This is the first geometry object \t\t",mesh.scene.nodes[0])



This is the whole scene source list object \(1-cos) <Collada geometries=1>
This is the first geometry object \(1-cos)\(1-cos) <Geometry id=Cube_001-mesh, 1 primitives>
This is the first geometry object 		 <Node transforms=1, children=1>


# Color filtering


### Function


In [3]:
def filterByMaterial(scene, materialName : str = "Material.001", returnBlanks : bool = False,  ) -> list:
    """A function that returns a list of collada.geometry.BoundGeometry objects. Checks the first node of given obj.primitives() for material. Possbile to use obj.original on obj from list to rerutn not-bound geometry.
        'cl' as if collada imported as 'cl'.
        yellowMaterial = 'Material.001' ;
        blueMaterial = 'Material.002'
    Args:
        scene (cl.collada.scene, <class 'collada.scene.Scene'>): A cl.Collada object. (e.g. read from file)
        materialName (str, optional): Name of the material that cointains the required color/attribute. Defaults to "Material.001".
        returnBlanks (bool, optional): If True then returns list with notbound geometry without materials. Defaults to False.

    Returns:
        list: List of collada.geometry.BoundGeometry objects with selected material
    """
    
    if (type(scene) != cl.scene.Scene):
        print("Incorrect type as scene!")
        raise ValueError
    
    listOf_sceneGeoNodes = list(scene.objects('geometry'))
    yellowMaterial = 'Material.001'
    blueMaterial = 'Material.002'

    listOf_yellows  :list = []
    listOf_blues    :list = []
    listOf_None     :list = []

    # TODO check if idx can be used for proper node.id / cube.id assignment
    for idx, obj in enumerate(listOf_sceneGeoNodes):
        geo = list(obj.primitives())
        triangle = geo[0]
        if triangle.material is None:
            # print(type(triangle.material))
            listOf_None.append(obj)
            continue
        
        # print(triangle.material.name)
        if (triangle.material.name == yellowMaterial):
            listOf_yellows.append(obj)
        else:
            listOf_blues.append(obj)
            
        if (returnBlanks == True):
            print("Returned listOf_None",  len(listOf_None), "objects")
            return listOf_None
        elif (materialName == yellowMaterial):
            print("Returned listOf_yellows",  len(listOf_yellows), "objects")
            return listOf_yellows
        else:
            print("Returned listOf_blues",  len(listOf_blues), "objects")
            return listOf_blues

# Filtering by area xyz (0;15)

### Function

In [4]:
def filterByUniformArea(scene, upper_bound : float, lower_bound : float = 0, inside : bool = True, verbose : bool = False) -> list:
    """_summary_

    Args:
        scene (_type_): _description_
        upper_bound (float): _description_
        lower_bound (float, optional): _description_. Defaults to 0.
        inside (bool, optional): _description_. Defaults to True.
        verbose (bool, optional): _description_. Defaults to False.

    Returns:
        list: _description_
    """
    
    listOf_inArea :list = []
    listOf_outOfBounds :list = []
    listOf_sourceGeoNodes = list(scene.objects('geometry'))
    
    for idx_1, obj in enumerate(listOf_sourceGeoNodes):
        geo = list(obj.primitives())
        triangle = geo[0]
        
        for  idx_2 , arrayOfVerts in enumerate(triangle.vertex):
            vertInArea = np.all((lower_bound < arrayOfVerts) & (arrayOfVerts < upper_bound)) 
            if (vertInArea == True):
                listOf_inArea.append(obj)
                break
            if (idx_2 == len(triangle.vertex) -1 ):
                listOf_outOfBounds.append(obj)
        if (verbose == True):
            print("This is the amount of inArea",           len(listOf_inArea))
            print("This is the amount of outOfBounds",      len(listOf_outOfBounds))
    if (inside == True):
        return listOf_inArea
    else:
        return listOf_outOfBounds
        

## Transforming the scene
### Problems
- MatrixTransform sucks
- How to rotate the whole set? - Best solution would be to place them under one node, calculate the center and rotate the parent?

# Creating new objects

### Function

In [5]:
def prepareColladaObj(obj, objectToCopyFrom, list_to_pick_from, nameOfTheScene : str = "myscene" ):
    """_summary_

    Args:
        obj (_type_): _description_
        objectToCopyFrom (_type_): _description_
        list_to_pick_from (_type_): _description_
        nameOfTheScene (str, optional): _description_. Defaults to "myscene".

    Returns:
        _type_: _description_
    """
    
    # TODO Correct amount of nodes
    amountOfObjects = len(list(objectToCopyFrom.scene.objects('geometry')))
    for x_Eff in objectToCopyFrom.effects:
        obj.effects.append(x_Eff)
        
    for x_Mat in objectToCopyFrom.materials:
        obj.materials.append(x_Mat)
        
    indexOfCorrectNodes = []
    for x_Geo in list_to_pick_from:
        x_Geo = x_Geo.original
        print("x_Geo", x_Geo.name)
        indexOfCorrectNodes.append(amountOfObjects -  int(x_Geo.name[-3:]))
        newSceneMesh.geometries.append(x_Geo)
    print("indexOfCorrectNodes", indexOfCorrectNodes)
    print("objectToCopyFrom.scene.nodes", objectToCopyFrom.scene.nodes)
    compressed_nodes = [val for idx, val in enumerate(objectToCopyFrom.scene.nodes) if idx in indexOfCorrectNodes]
    print(compressed_nodes[0].id)
    myscene = cl.scene.Scene(nameOfTheScene, compressed_nodes)
    obj.scenes.append(myscene)
    obj.scene = myscene
    
    return obj

In [6]:
newSceneMesh = cl.Collada()
list_to_pick_from = filterByMaterial(scene = mesh.scene, materialName= "lol")
print(list_to_pick_from)
newSceneMesh = prepareColladaObj(newSceneMesh, mesh, list_to_pick_from= list_to_pick_from)
newSceneMesh.assetInfo.upaxis= "Z_UP"


Returned listOf_blues 1 objects
[<BoundGeometry id=Cube_001-mesh, 1 primitives>]
x_Geo Cube_001
indexOfCorrectNodes [0]
objectToCopyFrom.scene.nodes [<Node transforms=1, children=1>]
Cube_parentNode0


# Calculate mean of points

### Function

In [7]:
def meanOfCentre(list_to_pick_from : list, verbose : bool = True) -> np.ndarray:
    """_summary_

    Args:
        list_to_pick_from (list): _description_
        verbose (bool, optional): _description_. Defaults to True.

    Returns:
        np.ndarray: _description_
    """
    
    centralPoint : np.array = 0
    tmpSum = 0
    amount : int = 0
    for x_Geo in list_to_pick_from:
        geo = list(x_Geo.primitives())
        triangle = geo[0]
        
        for  idx_2 , arrayOfVerts in enumerate(triangle.vertex):
            tmpSum += arrayOfVerts
            amount += 1
    centralPoint = tmpSum/amount
            
    if (verbose == True):
        print("Sum of the verts:\n\t", tmpSum)
        print("Amount of the verts:\n\t", amount)
        print("Central mean of vertices:\n\t", centralPoint)
    
    return centralPoint

In [8]:
originPoint = meanOfCentre(list_to_pick_from)

Sum of the verts:
	 [0. 0. 0.]
Amount of the verts:
	 8
Central mean of vertices:
	 [0. 0. 0.]


# Applying transformations


### Function

In [9]:
def applyTransformation(scene, pivotOfRotation = [0,0,0] ,
                        transform_offsetX:float = 0,
                        transform_offsetY:float = 0,
                        transform_offsetZ:float = 0,
                        scalevalueX:float = 1,
                        scalevalueY:float = 1,
                        scalevalueZ:float = 1,
                        degreesToRotate : float = 0,
                        verbose : bool = False ) -> None:
    """_summary_

    Args:
        scene (_type_): _description_
        pivotOfRotation (_type_): _description_
        transform_offsetX (float, optional): _description_. Defaults to 0.
        transform_offsetY (float, optional): _description_. Defaults to 0.
        transform_offsetZ (float, optional): _description_. Defaults to 0.
        degreesToRotate (float, optional): _description_. Defaults to 0.
    """
# http://www.opengl-tutorial.org/assets/faq_quaternions/index.html#Q11

    matrixScale = cl.scene.ScaleTransform(scalevalueX, scalevalueY, scalevalueZ)
    # matrixRotation = cl.scene.RotateTransform(pivotOfRotation[0], pivotOfRotation[1], pivotOfRotation[2], math.radians(degreesToRotate))
    
    cos = np.cos(np.radians(degreesToRotate))
    sin = np.sin(np.radians(degreesToRotate))
    # matrixRotation = np.array([[cos, -sin, 0, 0] ,
    #                           [sin, cos, 0, 0]  ,
    #                           [0,0,1,0]         ,
    #                           [0,0,0,1]         ])
    x,y,z = pivotOfRotation[0], pivotOfRotation[1], pivotOfRotation[2]
    matrixRotation = np.array([
            [(1-cos) * x * x + cos,
             (1-cos) * x * y - sin * z,
             (1-cos) * x * z + sin * y,
             0],
            [(1-cos) * x * y + sin * z,
             (1-cos) * y * y + cos,
             (1-cos) * z * y - sin * x,
             0],
            [(1-cos) * x * z - sin * y,
             (1-cos) * y * z + sin * x,
             (1-cos) * z * z + cos,
             0],
            [0, 0, 0, 1],
        ])
    
    
    
    matrixTranslate = cl.scene.TranslateTransform(transform_offsetX, transform_offsetY, transform_offsetZ)
    
    for idx, nodeX in enumerate(scene.nodes):
        transforms_old = nodeX.transforms[0]
        old_node = scene.nodes[idx]
        
        # final_matrix = cl.scene.MatrixTransform(np.dot(np.dot(matrixRotation.matrix, matrixTranslate.matrix), transforms_old.matrix).flatten())
        # ? T x R x S
        # ?TransformedVector = TranslationMatrix * RotationMatrix * ScaleMatrix * OriginalVector;
        # final_matrix = cl.scene.MatrixTransform(np.dot(np.dot(np.dot(matrixTranslate.matrix,matrixRotation.matrix), matrixScale.matrix), transforms_old.matrix).flatten())
        final_matrix = cl.scene.MatrixTransform(np.dot(np.dot(np.dot(matrixTranslate.matrix,matrixRotation), matrixScale.matrix), transforms_old.matrix).flatten())
        # new_node = cl.scene.Node(old_node.id, old_node.children, [])
        scene.nodes[idx].transforms[0] = final_matrix
        (scene.nodes[idx]).save()
    if (verbose == True):
        print("Scaling _>\n",matrixScale.matrix)
        # print("Rotations _>\n",matrixRotation.matrix)
        print("Rotations _>\n",matrixRotation)
        # print("OLD _>\n", transforms_old.matrix)
        print("NEW _>\n", scene.nodes[0].transforms[0].matrix)

In [14]:
# ! OH MY FCKING GOD
# matrixRotation = cl.scene.RotateTransform(originPoint[0], originPoint[1], originPoint[2], math.radians(20))
# matrixTranslate = cl.scene.TranslateTransform(0, 100, 0)

# applyTransformation(scene= newSceneMesh.scene, transform_offsetY= 5,scalevalueX = 1, scalevalueY = 2, scalevalueZ = 1, verbose= True)
# applyTransformation(scene= newSceneMesh.scene, scalevalueX = 1, scalevalueY = 1.5, scalevalueZ = 1, verbose= True)

listOf_objectsAfterTransform = list(newSceneMesh.scene.objects('geometry'))

newPivot = meanOfCentre(listOf_objectsAfterTransform)
print("NewPivot = ", newPivot)
# ?Change of pivot also affects scale?
# ?What is lacking is another function that would take the current matrix and apply transforms at LOCAL space, not GLOBAL
applyTransformation(scene= newSceneMesh.scene, pivotOfRotation = [0,0,1], degreesToRotate=20, verbose= True)

# applyTransformation(scene= newSceneMesh.scene, pivotOfRotation= newPivot, degreesToRotate= 30 ,  verbose= True)

print("Mesh old transform\n", mesh.scene.nodes[0].transforms[0].matrix)

Sum of the verts:
	 [-13.68080573  37.58770483   0.        ]
Amount of the verts:
	 8
Central mean of vertices:
	 [-1.71010072  4.6984631   0.        ]
NewPivot =  [-1.71010072  4.6984631   0.        ]
Scaling _>
 [[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]
Rotations _>
 [[ 0.93969262 -0.34202014  0.          0.        ]
 [ 0.34202014  0.93969262  0.          0.        ]
 [ 0.          0.          1.          0.        ]
 [ 0.          0.          0.          1.        ]]
NEW _>
 [[ 0.76312941 -1.28557522  0.06676517 -3.21393805]
 [ 0.64034161  1.53208889  0.05602263  3.83022222]
 [-0.08715574  0.          0.9961947   0.        ]
 [ 0.          0.          0.          1.        ]]
Mesh old transform
 [[ 0.76312941 -1.28557522  0.06676517 -3.21393805]
 [ 0.64034161  1.53208889  0.05602263  3.83022222]
 [-0.08715574  0.          0.9961947   0.        ]
 [ 0.          0.          0.          1.        ]]


In [15]:
newSceneMesh.write("test0Pos.dae")